In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.resolve()

In [ ]:
using Turing, Optim, SPTLikelihoods
using ForwardDiff
using Capse
using JSON
using BenchmarkTools
using NPZ
using MicroCanonicalHMC
using LinearAlgebra
using MCMCDiagnosticTools
import MCMCChains: compute_duration
include("utils.jl")

In [ ]:
weights_folder = "../data/weights/weights_cosmopowerspace_10000/"

emu_TT = Capse.load_emulator(weights_folder*"TT/")#, Capse.SimpleChainsEmulator)
emu_EE = Capse.load_emulator(weights_folder*"EE/")#, Capse.SimpleChainsEmulator)
emu_TE = Capse.load_emulator(weights_folder*"TE/")#, Capse.SimpleChainsEmulator)
emu_PP = Capse.load_emulator(weights_folder*"PP/");#, Capse.SimpleChainsEmulator);

In [ ]:
function call_emu_planck(θ, Emu_TT, Emu_TE, Emu_EE)
    return vcat(0, Capse.get_Cℓ(θ, Emu_TT)[1:3199]),
           vcat(0, Capse.get_Cℓ(θ, Emu_TE)[1:3199]),
           vcat(0, Capse.get_Cℓ(θ, Emu_EE)[1:3199])
end

In [ ]:
theory(θ) = call_emu_planck(θ, emu_TT, emu_TE, emu_EE)

In [ ]:
x = rand(6)
@benchmark theory($x)

In [ ]:
a, b, c = theory([3.034, 0.97, 68.3, 0.02224, 0.1166, 0.054])

In [ ]:
model_matrix = SPTLikelihoods.compute_theory(a,b,c, 0.00,
    62.61, 27.9, 24.3, 16.7, 28.6, 78.5,
    0.041, 0.0177, 0.0157, 0.0115, 0.0188, 0.048,
    1.93, -2.53, 1.48, 5.2, 0.8, 1.85, 4.7, 0.09,
    3.7, 0.052, -2.42, 1.51, 0.138, -2.42, 1.51,
    1., 0.9975, 0.9930, 1.0009, 1.002, 1.019, 3200)
Σ = SPTLikelihoods.compute_cov(model_matrix)

Γ = sqrt(Σ)
iΓ = inv(Γ);

In [ ]:
@model function SPT3G_TTTEEE(data, iΓ)
    # cosmo params
    ln10As ~ Uniform(0.25, 0.35)
    ns     ~ Uniform(0.88, 1.06)
    h0     ~ Uniform(0.60, 0.80)
    ωb     ~ Uniform(0.1985, 0.25)
    ωc     ~ Uniform(0.08, 0.20)
    τ      ~ Normal(0.065, 0.015)
    
    θ = [10*ln10As, ns, 100*h0, ωb/10, ωc, τ]
    Dl_TT, Dl_TE, Dl_EE = theory(θ)
    
    κ100 ~ Normal(0, 0.045)
    κ = κ100/100
    
    D_TT_90_90 ~ Normal(51.3, 9.4)# 62.61
    D_TT_90_150 ~ Normal(22.4, 7.1)#27.9
    D_TT_90_220 ~ Normal(20.7, 5.9)#24.3
    D_TT_150_150 ~ Normal(15.3, 4.1)#16.7
    D_TT_150_220 ~ Normal(28.4, 4.2)#28.6
    D_TT_220_220 ~ Normal(76.0, 14.9)# 78.5
    
    D_EE_90_90 ~ Normal(0.041, 0.012)# 0.041
    D_EE_90_150 ~ Normal(0.0180, 0.0054)#0.0177
    D_EE_90_220 ~ Normal(0.0157, 0.0047)# 0.0157
    D_EE_150_150 ~ Normal(0.0115, 0.0034)#0.0115
    D_EE_150_220 ~ Normal(0.0190, 0.0057)# 0.0188
    D_EE_220_220 ~ Normal(0.048, 0.014)#0.048
    
    A_80_cirrus ~ Normal(1.88, 0.48)#1.93
    α_cirrus ~ Normal(-2.53, 0.05)#-2.53
    β_cirrus ~ Normal(1.48, 0.02)#1.48
    
    A_80_cib ~ Normal(3.2, 1.8)#5.2
    α_cib = 0.8
    β_cib ~ Normal(2.26, 0.38)#1.85
    
    A_tSZ ~ Normal(3.2, 2.4)#4.7
    ξ_tsz_CIB ~ Normal(0.18, 0.33)#0.09
    A_kSZ ~ Normal(3.7, 4.6)#3.7
    
    A_80_EE ~ Normal(0.05, 0.022)#0.052
    α_EE ~ Normal(-2.42, 0.04)#-2.42
    β_EE ~ Normal(1.51, 0.04)#1.51
    
    A_80_TE ~ Normal(0.12, 0.051)#0.138
    α_TE ~ Normal(-2.42, 0.04)#-2.42
    β_TE ~ Normal(1.51, 0.04)#1.51
    
    T_E_cal ~ MvNormal(ones(6), SPTLikelihoods.cal_cov)
    cal_T_90, cal_T_150, cal_T_220, cal_E_90, cal_E_150, cal_E_220 = T_E_cal
    
    model_matrix = SPTLikelihoods.compute_theory(Dl_TT, Dl_TE, Dl_EE, κ,
    D_TT_90_90, D_TT_90_150, D_TT_90_220, D_TT_150_150, D_TT_150_220, D_TT_220_220,
    D_EE_90_90, D_EE_90_150, D_EE_90_220, D_EE_150_150, D_EE_150_220, D_EE_220_220,
    A_80_cirrus, α_cirrus, β_cirrus, A_80_cib, α_cib, β_cib, A_tSZ, ξ_tsz_CIB,
    A_kSZ, A_80_EE, α_EE, β_EE, A_80_TE, α_TE, β_TE,
    cal_T_90, cal_T_150, cal_T_220, cal_E_90, cal_E_150, cal_E_220, 3200)
    
    vec_residuals = iΓ * SPTLikelihoods.slice_theory(model_matrix)
    
    data ~ MvNormal(vec_residuals, I)
    
    return nothing
    
end

SPT3G_TTTEEE_model = SPT3G_TTTEEE(zeros(728), iΓ);

In [ ]:
bestfit_SPT = optimize(SPT3G_TTTEEE_model, MAP(), Optim.Options(iterations=1000000, allow_f_increases=true, show_trace=true))

In [ ]:
chains_SPT_NUTS = sample(SPT3G_TTTEEE_model, NUTS(500, 0.65), 2000, init_params=bestfit_SPT.values.array)

In [ ]:
npzwrite("chains_NUTS_SPT.npy", chains_SPT_NUTS.value.data[:,:,1])

In [ ]:
CPU_s_SPT_NUTS = compute_duration(chains_SPT_NUTS)
SPT_NUTS_ESS = mean(MCMCDiagnosticTools.ess_rhat(chains_SPT_NUTS)[chains_SPT_NUTS.name_map[:parameters],:ess])
SPT_NUTS_ESS_s = SPT_NUTS_ESS/CPU_s_SPT_NUTS

In [ ]:
SPT_NUTS_ESS

In [ ]:
d = length(bestfit_SPT.values.array)


target = TuringTarget(SPT3G_TTTEEE_model)
nadapts = 20_000
nsteps = 100_000
spl = MCHMC(nadapts, 0.001; init_eps=0.005, L=sqrt(d),# sigma=ones(d),
            adaptive=true)
@time spt_mchmc = Sample(spl, target, nsteps;
                       progress=true,
                       dialog=true, file_name="chain_1",
                       initial_x=bestfit_SPT.values.array);

In [ ]:
chains_SPT_MCHMC = mapreduce(permutedims, vcat, spt_mchmc)[:,1:39];

In [ ]:
mean(chains_SPT_MCHMC[:,1])

In [ ]:
npzwrite("chains_MCHMC_SPT.npy", chains_SPT_MCHMC)

In [ ]:
vec_ess = Summarize(spt_mchmc)[1][1:7]

In [ ]:
mean(Summarize(spt_mchmc)[1][1:34])

In [ ]:
mean(vec_ess)